In [1]:
import os
import torch
import torchvision
import torch.nn as nn
from datetime import datetime

from torch.utils.data import DataLoader
from torch.autograd import Variable

from LipReadDataTrain import ReadData
from LipNet import LipNet, LipSeqLoss

In [6]:
train_image_file = os.path.join("H:\\ZSF-Datasets\\新网银行唇语识别竞赛数据\\1.训练集\\lip_train")
train_label_file = os.path.join("H:\\ZSF-Datasets\\新网银行唇语识别竞赛数据\\1.训练集\\lip_train.txt")
training_dataset = ReadData(train_image_file, train_label_file, seq_max_lens=24)
training_data_loader = DataLoader(training_dataset, batch_size=20, shuffle=True, num_workers=12, drop_last=True)

# GPU
device = torch.device('cuda:0')
# # CPU 
# device = torch.device('cpu')

model = LipNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fc = LipSeqLoss().to(device)

In [7]:
for epoch in range(1,1000):
    print(epoch)
    model.train()
    for i_batch, sample_batched in enumerate(training_data_loader):
        
        input_data = Variable(sample_batched['volume']).to(device) 
        labels = Variable(sample_batched['label']).to(device)
        length = Variable(sample_batched['length']).to(device)
        
        optimizer.zero_grad()
        result = model(input_data)  
        loss = loss_fc(result, length, labels)   
        loss.backward()
        optimizer.step()
        
    if epoch % 5 == 0:
        current_time = datetime.now()
        print("current time:", current_time)
        print("number of epoch:", epoch)
        print("current loss:", loss)
                   
        # save model
        torch.save(model.state_dict(), "./weight/demo_net_epoch_{}.pt".format(epoch))

1


error: Caught error in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Users\zsf\Anaconda3\envs\tf-gpu\lib\site-packages\torch\utils\data\_utils\worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "C:\Users\zsf\Anaconda3\envs\tf-gpu\lib\site-packages\torch\utils\data\_utils\fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Users\zsf\Anaconda3\envs\tf-gpu\lib\site-packages\torch\utils\data\_utils\fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Users\zsf\PycharmProjects\lip_reading_demo\lip_reading_demo1\LipReadDataTrain.py", line 37, in __getitem__
    frames_ = [cv2.cvtColor(img, cv2.COLOR_BGR2RGB) for img in frames]
  File "C:\Users\zsf\PycharmProjects\lip_reading_demo\lip_reading_demo1\LipReadDataTrain.py", line 37, in <listcomp>
    frames_ = [cv2.cvtColor(img, cv2.COLOR_BGR2RGB) for img in frames]
cv2.error